In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from time import time

class Args():
    gamma = 0.99
    seed = 543
    render = False
    log_interval = 10

args = Args()

env = gym.make('CartPole-v1')
env.seed(args.seed)
torch.manual_seed(args.seed)

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


def finish_episode():
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        R = r + args.gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

t0 = time()

def run():
    running_reward = 10
    for i_episode in count(1):
        state, ep_reward = env.reset(), 0
        for t in range(1, 10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if args.render:
                env.render()
            policy.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        if i_episode % args.log_interval == 0:
            print('{:4.1f} s, Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  time() - t0, i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [2]:
run()     

 0.2 s, Episode 10	Last reward: 14.00	Average reward: 13.93
 0.3 s, Episode 20	Last reward: 32.00	Average reward: 21.35
 0.5 s, Episode 30	Last reward: 38.00	Average reward: 30.83
 0.8 s, Episode 40	Last reward: 35.00	Average reward: 42.31
 1.0 s, Episode 50	Last reward: 60.00	Average reward: 44.31
 1.3 s, Episode 60	Last reward: 46.00	Average reward: 54.73
 1.6 s, Episode 70	Last reward: 60.00	Average reward: 63.02
 1.9 s, Episode 80	Last reward: 99.00	Average reward: 66.24
 2.0 s, Episode 90	Last reward: 33.00	Average reward: 49.81
 2.1 s, Episode 100	Last reward: 23.00	Average reward: 40.63
 2.3 s, Episode 110	Last reward: 35.00	Average reward: 38.45
 2.7 s, Episode 120	Last reward: 138.00	Average reward: 55.04
 3.3 s, Episode 130	Last reward: 228.00	Average reward: 98.41
 4.5 s, Episode 140	Last reward: 500.00	Average reward: 180.25
 5.6 s, Episode 150	Last reward: 31.00	Average reward: 206.03
 6.1 s, Episode 160	Last reward: 108.00	Average reward: 169.72
 6.6 s, Episode 170	Last r